# Creating first agent

## Install Packages

In [2]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import the libraries

In [14]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [15]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Create the agent

In [16]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_YVxnuANECcPWEGWNRrFf8bLD


## Create thread

In [17]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_AWb6CyHej0YEVxnozmz8toCp


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [24]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        # Fetch the first message in the thread to see the agent's response
        messages = list(project_client.agents.messages.list(thread_id=thread.id))
        if messages:
            first_message = messages[0]
            content = first_message['content']
            # Extract the text value if present
            if isinstance(content, list) and content:
                text_value = content[0].get('text', {}).get('value', '')
                print(text_value)

## Run the agent

In [27]:
user_input = "What are the 5 things I can see in New York?"
run_agent(user_input)

Created message, ID: msg_PinJeeiDlUresfR47kndgNRl
Run finished with status: completed
Certainly! Here are five must-see attractions in New York City:

1. **Statue of Liberty and Ellis Island**: Visit these iconic landmarks to learn about the history of immigration and enjoy stunning views of New York Harbor. The Statue of Liberty is a universal symbol of freedom, while Ellis Island offers a rich, immersive historical experience.

2. **Central Park**: This expansive green space in the heart of Manhattan offers something for everyone, including walking trails, boating, picturesque bridges, historical statues, and the Central Park Zoo. It’s a perfect place to relax and enjoy nature amidst the urban environment.

3. **Times Square**: Known for its buzzing atmosphere, bright lights, massive billboards, and Broadway theaters, Times Square is the epitome of New York City’s vibrant energy. It's a prime location for entertainment and people-watching.

4. **Empire State Building**: Ascend to the

## Cleanup resources

In [28]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent


## Creating an agent and thread

In [31]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )
print(f"Created agent, ID: {agent.id}")
agent_id = agent.id


Created agent, ID: asst_dXEhyPlIdi0nnGCCKUDva3jT


## Retrieving an agent

In [32]:
agent = project_client.agents.get_agent(agent_id)

## Create thread

To retrieve a thread
- thread = project.agents.threads.get("")

In [33]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_qopPGZiB61t2jPDyMk1aNqr9


## Run agent

In [34]:
user_input = "What are the 5 things I can see in San Francisco?"
run_agent(user_input)

Created message, ID: msg_0IIEBPmLxc0nHjUTr6Uk1fKM
Run finished with status: completed
San Francisco is a diverse city with many attractions. Here are five must-see places:

1. **Golden Gate Bridge**: One of the most iconic landmarks in San Francisco, offering breathtaking views and photo opportunities. You can walk, bike, or drive across the bridge.
   
2. **Alcatraz Island**: This historic former prison is located on an island in San Francisco Bay. Take a ferry ride to visit the abandoned prison cells and learn about its history on a guided tour.
   
3. **Fisherman's Wharf**: A popular waterfront area known for its seafood, street performers, souvenir shops, and family-friendly attractions. Don’t miss a visit to Pier 39 to see the sea lions.
   
4. **Chinatown**: San Francisco's Chinatown is the oldest and one of the largest in North America. Explore its bustling streets, traditional shops, temples, and enjoy authentic Chinese cuisine.
   
5. **Golden Gate Park**: This vast urban park

## Cleanup resources

In [35]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
